# LLM Iteration 04

**Aim**

Update the code output from iteration 3 so a random patient type is not admitted each time, but different patient types arrive independently of each other. It should also be able to input a different random seed for each patient type for reproducibility and testing.

**Prompt**

```python
import simpy
import numpy as np

NUM_BEDS = 10
LENGTH_OF_STAY = 7
RUN_LENGTH = 60  # Default run length in days (2 months)
INTERARRIVAL_TIMES = {
    'Stroke': 1.2,
    'TIA': 9.3,
    'Complex Neurological': 3.6,
    'Other': 3.2
}

class StrokeWard:
    def __init__(self, env):
        self.env = env
        self.beds = simpy.Resource(env, capacity=NUM_BEDS)
        self.patient_count = 0

    def admit_patient(self, patient):
        with self.beds.request() as bed:
            yield bed
            print(f"Patient {patient.id} of type {patient.type} admitted at day {round(self.env.now, 2)}")
            yield self.env.timeout(LENGTH_OF_STAY)
            print(f"Patient {patient.id} of type {patient.type} discharged at day {round(self.env.now, 2)}")

class Patient:
    def __init__(self, id, type):
        self.id = id
        self.type = type

def patient_generator(env, ward, interarrival_time, random_seed=None):
    rng = np.random.default_rng(random_seed)
    while True:
        yield env.timeout(rng.exponential(interarrival_time))
        patient_type = np.random.choice(list(INTERARRIVAL_TIMES.keys()))
        patient = Patient(ward.patient_count, patient_type)
        ward.patient_count += 1
        env.process(ward.admit_patient(patient))

env = simpy.Environment()
ward = StrokeWard(env)
env.process(patient_generator(env, ward, INTERARRIVAL_TIMES['Stroke'], random_seed=42))
env.run(until=RUN_LENGTH)
```

Update this code to use the correct interarrival time for each patient type and have the option to input a different random seed for each patient type.

**LLM Output**

In [1]:
import simpy
import numpy as np

In [2]:
NUM_BEDS = 10
LENGTH_OF_STAY = 7
RUN_LENGTH = 60  # Default run length in days (2 months)
INTERARRIVAL_TIMES = {
    'Stroke': 1.2,
    'TIA': 9.3,
    'Complex Neurological': 3.6,
    'Other': 3.2
}

In [3]:
class StrokeWard:
    def __init__(self, env):
        self.env = env
        self.beds = simpy.Resource(env, capacity=NUM_BEDS)
        self.patient_count = 0

    def admit_patient(self, patient):
        with self.beds.request() as bed:
            yield bed
            print(f'Patient {patient.id} of type {patient.type} admitted at '
                  f'day {round(self.env.now, 2)}')
            yield self.env.timeout(LENGTH_OF_STAY)
            print(f'Patient {patient.id} of type {patient.type} discharged at '
                  'day {round(self.env.now, 2)}')

class Patient:
    def __init__(self, id, type):
        self.id = id
        self.type = type

def patient_generator(env, ward, interarrival_time, random_seed=None):
    rng = np.random.default_rng(random_seed)
    while True:
        yield env.timeout(rng.exponential(interarrival_time))
        patient_type = np.random.choice(list(INTERARRIVAL_TIMES.keys()),
                                        p=[INTERARRIVAL_TIMES[key] for key \
                                           in INTERARRIVAL_TIMES])
        patient = Patient(ward.patient_count, patient_type)
        ward.patient_count += 1
        env.process(ward.admit_patient(patient))

In [4]:
env = simpy.Environment()
ward = StrokeWard(env)

# Update to allow different random seeds for each patient type
random_seeds = {'Stroke': 42, 'TIA': 24, 'Complex Neurological': 56,
                'Other': 78}
for patient_type in INTERARRIVAL_TIMES:
    env.process(patient_generator(env, ward, INTERARRIVAL_TIMES[patient_type],
                                  random_seed=random_seeds[patient_type]))

#env.run(until=RUN_LENGTH)

**Reflection**

Running this code outputs an error so has been commented out. The issue is the sampling of the patient type in the function patient_generator(), where it uses np.random.choice and assumes that the interarrival times specified are probabilities of that patient arriving instead of the mean interarrival time. This is because it was not passed all the information about the model set up. In the next test, the initial prompt will also be given in order to improve the LLM's understanding of the problem.